# 신경망으로 챗봇 만들기

훈련 데이터를 만들고 신경망을 훈련시킨 다음 훈련된 모델을 사용하여 챗봇을 만들 것이다.

먼저, 필수 라이브러리를 설치하기로 했다.

In [24]:
!pip install numpy scipy
!pip install scikit-learn
!pip install pillow
!pip install h5py

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [25]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install tensorflow-gpu

Defaulting to user installation because normal site-packages is not writeable
  Running setup.py clean for tensorflow-gpu
Failed to build tensorflow-gpu
    Running setup.py install for tensorflow-gpu: started
    Running setup.py install for tensorflow-gpu: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\USER\\AppData\\Local\\Temp\\pip-install-yc331n0x\\tensorflow-gpu_9c9b838ad77f41ffb6ac2c459f029ac0\\setup.py'"'"'; __file__='"'"'C:\\Users\\USER\\AppData\\Local\\Temp\\pip-install-yc331n0x\\tensorflow-gpu_9c9b838ad77f41ffb6ac2c459f029ac0\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\USER\AppData\Local\Temp\pip-wheel-gpjic88g'
       cwd: C:\Users\USER\AppData\Local\Temp\pip-install-yc331n0x\tensorflow-gpu_9c9b838ad77f41ffb6ac2c459f029ac0\
  Complete output (17 lines):
  Traceback (most recent call last):
    File "<string>", line 1, in <module>
    File "C:\Users\

In [4]:
!pip install keras

Defaulting to user installation because normal site-packages is not writeable


# 1. 라이브러리 설치

우선 이 신경망 구동 챗봇에 필요한 라이브러리를 설치해보았다.
Keras는 백엔드에서 텐서플로우(다른 하위 레벨 기계 학습 라이브러리) 를 활용하는 기계 학습 라이브러리이다. 이렇게 케라스를 쓰면 심층 신경망으로 챗봇을 만들 수 있다.

In [6]:
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import SGD
from keras.layers import Dense
 
from numpy import argmax
import numpy as np
import re

# 2. 입력 훈련 데이터

먼저 챗봇에 대한 다음 교육 데이터를 포함해야한다.
1. X는 사용자가 입력할 수 있는 다양한 입력을 나타내고,
2. Y는 입력의 의도를 나타낸다.

In [26]:
X = ['Hi',
     'Hello',
     'How are you?',
     'I am making',
     'making',
     'working',
     'studying',
     'see you later',
     'bye',
     'goodbye']

In [27]:
print(len(X))

10


In [28]:
Y = ['greeting',
     'greeting',
     'greeting',
     'busy',
     'busy',
     'busy',
     'busy',
     'bye',
     'bye',
     'bye']

In [10]:
print(len(Y))

10


여기서 비슷한 의도를 가진 여러 개의 다른 문장들이 있다는 것을 주목해야한다. 아직 여기서는 3개의 의도(greeting, busy, bye)만 있지만 프로젝트에 원하는 만큼 추가할 수 있다. 그냥 쉽게 하기 위해서 3개만 넣어뒀다.

챗봇이 작동하는 방식은...
1. 입력 문장으로부터, 우리는 훈련된 AI 모델을 사용하여 의도를 확인한다.
2. 각 의도에 대해, 준비 된 응답을 가지고있는 것이다.

예를 들어, 입력의 의도가 인사말에 대한 것임을 확인하면 챗봇에 '안녕하세요'또는 '어떻게 지내십니까?'와 같은 인사말로 응답하도록 요청할 수 있다.

기계 학습을 사용하여 입력 문장을 다른 의도로 분류 할 수있는 모델을 만들 것이다. 
그러면 다음과 같이 만들어야한다..
1. 문장과 그 의도에 대한 목록을 포함하고 있는 훈련 데이터(위의 X및  Y)를 작성한다.
2. 훈련 데이터를 사용하여 분류기를 훈련한다. 
3. 입력 문장을 벡터화하고 분류기를 사용하여 의도를 결정한다. 

# 3. 텍스트 처리

평소와 같이 텍스트 처리부터 시작해야한다.

## 3.1 알파벳과 숫자가 아닌 문자 제거

In [29]:
def remove_non_alpha_numeric_characters(sentence):
    new_sentence = ''
    for alphabet in sentence:
        if alphabet.isalpha() or alphabet == ' ':
            new_sentence += alphabet
    return new_sentence

In [30]:
def preprocess_data(X):
    X = [data_point.lower() for data_point in X]
    X = [remove_non_alpha_numeric_characters(
        sentence) for sentence in X]
    X = [data_point.strip() for data_point in X]
    X = [re.sub(' +', ' ',
                data_point) for data_point in X]
    return X

In [13]:
X = preprocess_data(X)

vocabulary = set()
for data_point in X:
    for word in data_point.split(' '):
        vocabulary.add(word)

vocabulary = list(vocabulary)

## 문서 벡터 생성

In [14]:
X_encoded = []

def encode_sentence(sentence):
    sentence = preprocess_data([sentence])[0]
    sentence_encoded = [0] * len(vocabulary)
    for i in range(len(vocabulary)):
        if vocabulary[i] in sentence.split(' '):
            sentence_encoded[i] = 1
    return sentence_encoded

X_encoded = [encode_sentence(sentence) for sentence in X]

In [15]:
classes = list(set(Y))

Y_encoded = []
for data_point in Y:
    data_point_encoded = [0] * len(classes)
    for i in range(len(classes)):
        if classes[i] == data_point:
            data_point_encoded[i] = 1
    Y_encoded.append(data_point_encoded)

# 4. 훈련 데이터 및 테스트 데이터 생성

In [16]:
X_train = X_encoded
y_train = Y_encoded
X_test = X_encoded
y_test = Y_encoded

훈련 및 테스트 데이터에 사용하는 데이터 출력 및 확인

In [17]:
print (y_test)

[[1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0]]


In [18]:
print(len(X_train))

10


In [19]:
y_train

[[1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0]]

# 5. 모델 훈련

이제 훈련 데이터를 이용해 신경망을 훈련시키겠습니다.

In [20]:
model = Sequential()
model.add(Dense(units=64, activation='sigmoid',
                input_dim=len(X_train[0])))
model.add(Dense(units=len(y_train[0]), activation='softmax'))
model.compile(loss=categorical_crossentropy,
              optimizer=SGD(lr=0.01,
                            momentum=0.9, nesterov=True))
model.fit(np.array(X_train), np.array(y_train), epochs=100, batch_size=16)

Epoch 1/100


C:\Users\USER\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


1/1 [==============================] - 0s 411ms/step - loss: 1.1192
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 1.1093
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0982
Epoch 4/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0879
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0799
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0747
Epoch 7/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0719
Epoch 8/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0709
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0708
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 1.0707
Epoch 11/100
1/1 [==============================] - 0s 4ms/step - loss: 1.0702
Epoch 12/100
1/1 [==============================] - 0s 2ms/step - loss: 1.0690
Epoch 13/100
1/1 [==============================] - 0s 3ms/step - loss:

## 예측 목록 표시

In [21]:
predictions = [argmax(pred) for pred in model.predict(np.array(X_test))]

1/1 [==============================] - 0s 73ms/step


# 모델 평가

이제 모델을 평가해 봅시다. 모델에 의한 예측과 테스트 데이터를 비교할 것입니다:

In [22]:
correct = 0
for i in range(len(predictions)):
    if predictions[i] == argmax(y_test[i]):
        correct += 1

print ("Correct:", correct)
print ("Total:", len(predictions))

Correct: 7
Total: 10


# 챗봇 테스트

이제 챗봇을 테스트해 보겠습니다! 문장을 입력한 다음 신경망에서 예측되는 클래스를 확인합니다:

In [ ]:
while True:
    print ("Enter a sentence")
    sentence = input()
    prediction= model.predict(np.array([encode_sentence(sentence)]))
    print (classes[argmax(prediction)])

챗봇을 멈출 수 없다는 것을 알고 있습니까? 나중에 종료 명령을 추가해야 합니다(이전 노트를 참조하여 수행 방법을 확인하십시오.).

일단은 위의 중지 버튼(인터럽트 버튼)을 눌러서 챗봇을 중지하면 됩니다.

시도해 보세요. 정지 버튼을 누르고 상자에 뭔가를 입력해 보세요.

### 출처:
https://blog.eduonix.com/internet-of-things/simple-nlp-based-chatbot-python/